# CNN Example code
## Please install required module first
### You need to install opencv, numpy, matplotlib

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import sys
import tensorflow as tf
import keras
import pandas as pd
import sklearn as sk
import scipy as sp

import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: Linux-5.15.133.1-microsoft-standard-WSL2-build+-x86_64-with-glibc2.35
Tensor Flow Version: 2.15.0
Keras Version: 2.15.0

Python 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
Pandas 2.1.4
Scikit-Learn 1.3.2
SciPy 1.11.4
GPU is NOT AVAILABLE


2024-01-05 04:16:20.711888: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-05 04:16:20.712161: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Open file \& Read image

In [9]:
img_rows = 28
img_cols = 28

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

input_shape = (img_rows, img_cols, 1)

11490434/11490434 [==============================] - 1s 0us/step


In [10]:
for _i, _content in enumerate(y_train):
    if _i % 10 == 0 and _i != 0:
        print()
    print(_content, end=", ")

5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 
3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 
4, 0, 9, 1, 1, 2, 4, 3, 2, 7, 
3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 
1, 8, 7, 9, 3, 9, 8, 5, 9, 3, 
3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 
4, 4, 6, 0, 4, 5, 6, 1, 0, 0, 
1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 
9, 0, 2, 6, 7, 8, 3, 9, 0, 4, 
6, 7, 4, 6, 8, 0, 7, 8, 3, 1, 
5, 7, 1, 7, 1, 1, 6, 3, 0, 2, 
9, 3, 1, 1, 0, 4, 9, 2, 0, 0, 
2, 0, 2, 7, 1, 8, 6, 4, 1, 6, 
3, 4, 5, 9, 1, 3, 3, 8, 5, 4, 
7, 7, 4, 2, 8, 5, 8, 6, 7, 3, 
4, 6, 1, 9, 9, 6, 0, 3, 7, 2, 
8, 2, 9, 4, 4, 6, 4, 9, 7, 0, 
9, 2, 9, 5, 1, 5, 9, 1, 2, 3, 
2, 3, 5, 9, 1, 7, 6, 2, 8, 2, 
2, 5, 0, 7, 4, 9, 7, 8, 3, 2, 
1, 1, 8, 3, 6, 1, 0, 3, 1, 0, 
0, 1, 7, 2, 7, 3, 0, 4, 6, 5, 
2, 6, 4, 7, 1, 8, 9, 9, 3, 0, 
7, 1, 0, 2, 0, 3, 5, 4, 6, 5, 
8, 6, 3, 7, 5, 8, 0, 9, 1, 0, 
3, 1, 2, 2, 3, 3, 6, 4, 7, 5, 
0, 6, 2, 7, 9, 8, 5, 9, 2, 1, 
1, 4, 4, 5, 6, 4, 1, 2, 5, 3, 
9, 3, 9, 0, 5, 9, 6, 5, 7, 4, 
1, 3, 4, 0, 4, 8, 0, 4, 3, 6, 
8, 7, 6, 0, 9, 7, 5, 7, 2, 1, 
1, 6, 8, 9, 4, 1, 5, 2, 2, 9, 
0, 3, 9,

In [15]:
filter = np.array([[0,1,0],[0,1,1],[1,0,0]], dtype=np.uint8)
len(filter.shape)

2

In [ ]:
def conv_layer(filter, stride, padding):
    if len(filter.shape) != 2:
        raise Exception("Filter is not 2 dimension array")
    